In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS spire_catalog.spire_schema;
USE CATALOG spire_catalog;
USE SCHEMA spire_schema

In [0]:
def merge_with_table(table_name, temp_table):
    spark.sql(f"""MERGE INTO {table_name} s
                USING {temp_table} t
                ON s.file_id = t.file_id
                WHEN MATCHED THEN UPDATE SET *
                WHEN NOT MATCHED THEN INSERT *""")

In [0]:
##Get the last_modified_datetime from source table
def get_last_modified_datetime(table_name):
    last_datetime = spark.sql(f"SELECT MAX(last_modified_datetime) FROM {table_name}").collect()[0][0]
    return last_datetime

In [0]:
def create_chunked_table(table_fullname: str):
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_fullname} (
            id BIGINT GENERATED BY DEFAULT AS IDENTITY,
            file_id STRING,
            file_location STRING,
            file_name STRING,
            content STRING,
            created_datetime TIMESTAMP,
            last_modified_datetime TIMESTAMP
        )
        USING DELTA
        TBLPROPERTIES (delta.enableChangeDataFeed = true)
    """
    
    spark.sql(create_table_sql)

In [0]:
def build_permission_file_query(user_id: str) -> str:
    query = (
        "SELECT file_id "
        "FROM spire_catalog.spire_schema.sharepoint_permissions "
        f"WHERE user_id = '{user_id}'"
    )
    return query


In [0]:
def delete_file_id(table_name,file_ids):
    spark.sql(f"DELETE FROM {table_name} WHERE file_id IN ({file_ids})")